# Data Science: Bridging Principle and Practice
## Part 8: Model Selection (Bike Sharing case study)

<br/>

<div class="container">
    <div style="float:left;width:40%">
	    <img src="images/bikeshare_sun.jpg">
    </div>
    <div style="float:left;width:40%">
	    <img src="images/bikeshare_snow.PNG">
    </div>
</div>

### Table of Contents

[Case Study: Bike Sharing](#sectioncase)<br>

<ol start="8">
    <li><a href="#section8">Model Selection</a>
        <ol>
            <br>
            <li> <a href="#section8a">Using a Validation Set</a></li>
            <br>
            <li><a href="#section8b">Improving the Model</a></li>
        </ol>
    </li>
    </ol>


In [ ]:
# run this cell to import some necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
import seaborn as sns
from gofer.ok import check
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from scipy.linalg import lstsq
import ipywidgets as widgets
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from IPython.display import display,clear_output
from scripts.bike_model_widget import *

## Case Study: Capital Bike Share (review)<a id= "sectioncase"></a>

Bike-sharing systems have become increasingly popular worldwide as environmentally-friendly solutions to traffic congestion, inadequate public transit, and the "last-mile" problem. Capital Bikeshare runs one such system in the Washington, D.C. metropolitan area.

The Capital Bikeshare system comprises docks of bikes, strategically placed across the area, that can be unlocked by *registered* users who have signed up for a monthly or yearly plan or by *casual* users who pay by the hour or day. They collect data on the number of casual and registered users per hour and per day.

Let's say that Capital Bikeshare is interested in a **prediction** problem: predicting how many riders they can expect to have on a given day. [UC Irvine's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset) has combined the bike sharing data with information about weather conditions and holidays to try to answer this question.

In [ ]:
# run this cell to load the data
bikes = pd.read_csv("data/day_renamed_dso.csv", index_col=0)

# show the first five rows
bikes.head()

A few of the less straight-forward columns can be described as follows:
- **instant**: record index
- **is 2012** : 1 if the date is in 2012, 0 if the date is in 2011
- **is holiday** : 1 if day is a holiday, 0 otherwise
- **is work day** : 1 if day is not a weekend or holiday, otherwise 0
- **weather** :
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- **temp** : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
- **felt temp**: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
- **humidity**: Normalized humidity. The values are divided to 100 (max)
- **windspeed**: Normalized wind speed. The values are divided to 67 (max)
- **casual**: count of casual users
- **registered**: count of registered users
- **total riders**: count of total rental bikes (casual + registered)

## 8. Model Selection <a id="section8"></a>

### 8a. Using a Validation Set<a id="section8a"></a>
In Notebook 06 (Exploratory Data Analysis), we talk about the need to split our data into a *training set* and a *test set*. A test-train split reduces the chance of **overfitting**- that is, the chance that our model learns from the data so well that it makes incredibly accurate predictions for items in the training data, but makes wildly inaccurate predictions for data it hasn't been trained on. Another explanation of the test-train split and overfitting can be found at [this link](https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6).

In this notebook, we want to compare different linear models to one another to find a "best" model. One major indicator of a "best" model is how well it makes predictions for data it hasn't been trained on. 

A test set would be perfect for seeing how different models performs on new data to select the best model. But, when we compare multiple models to one another and make predictions for the test data multiple times, we get a new problem- we might overfit to the *test set*! In a perfect world, the test set is only ever used once, after you've selected your model, to get one final measure of how well it might generalize to new data.

We can get around this by using a **validation set**. A validation set is used during model selection. It acts a bit like a test set- the model isn't trained on data in the validation set, so having the model make predictions for the validation data can help us see how well our model performs on new data. There are several different methods of creating a validation set, but we'll use a simple split where we randomly select 20% of the rows in the training set.

Run the next cell to split the data into training, validation, and test sets. After this cell is run, 20% of the original data will be in the test set, 20% of the original data will be in the validation set, and the remaining 60% will be in the training set.

In [ ]:
# set the random seed
np.random.seed(28)

# set aside 20% of the data for testing
bikes, bike_test = train_test_split(bikes, train_size=0.8, test_size=0.2)

# set aside %25 of the remaining data for validation
bike_train, bike_val = train_test_split(bikes, train_size=0.75, test_size=0.25)

## 8b. Finding The Best Model <a id="section8b"></a>

In notebook 07, we created a linear regression model where we tried to predict the total number of riders on a given day based on the temperature, season, and whether or not the day was a work day. Ultimately, our model was not very accurate. Our challenge now is to build a better model.

Running the next cell will generate a widget that will train a model according to your chosen features and return several metrics for evaluating the model. 

<div class="alert alert-info">
<b>Note:</b> The code that trains the model and makes the graphs is somewhat lengthy and not important for the purposes of this assignment, so it has been put in a separate file. If you're interested, you can explore   "bike_model_widget.py" in the "scripts" folder. The code that creates the models and graphs is very similar to the code you saw and wrote in Notebook 07.
</div>

The widget contains the following:
- the grey rectangular buttons represent features that can be included in your linear model, as well as an intercept term. Clicking on one or more buttons will include those features in the model.
- the circular buttons let you choose which response variable you want your model to predict: total riders, registered riders, or casual riders. You may only choose one response variable.

Based on your selections, the widget will generate:
- the RMSE (root mean squared error) for the model's predictions for the training data and the validation data
- $\beta$:  the coefficients for your linear regression model
- scatter plots with best-fit lines for the predicted values plotted against the actual values, for both the training data (in blue) and the validation data (in gold)
- scatter plots with best-fit lines for the errors (residuals) plotted against the actual values of the response variable, for both the training data (in blue) and the validation data (in gold). These scatter plots also have black horizontal lines at zero to help you see whether the model tends to over- or under-estimate.

In [ ]:
# run this cell to generate the widget
display(expl_buttons,  intercept, response_radio,out)

<div class="alert alert-warning">
<b>EXERCISE:</b> <p>Use the widget to look for more accurate models. Try at least 10 combinations of features, and make sure to track which model seems to be performing the best.</p>
    <p>Note: it may take a few seconds for the graphs to update after you click the buttons.</p>
    <p>Record the parameters for your best model in the cell below. </p>
    <ol>
        <li><code>best_expl_vars</code> should be a list of the names of the explanatory variables (e.g. <code>["weather", "is holiday"]</code>)</li>
        <li><code>best_intercept</code> should be assigned to <code>True</code> if your best model included an intercept term, and <code>False</code> otherwise</li>
        <li><code>best_response_var</code> should be assigned to the name of the response variable used in your best model (e.g. <code>"casual riders"</code>)</li>
        <li><code>best_beta</code> should be assigned to the beta vector; the list of coefficients for your best model. The widget will print out this vector for each model</li>
    </ol>
</div>

In [ ]:
# list the explanatory variables used in your best model in a list
# variable names should be in quotation marks, and each name should be 
# separated by commas inside the square brackets
best_expl_vars = []

# replace the ellipses with True if your model used an intercept term
# otherwise, replace the ellipses with False
best_intercept = ...

# replace the ellipses with the name of the response variable
best_response_var = ...

# replace the ellipses with a list of the coefficients in the beta vector.
# you can copy and paste the beta values that the widget prints out, but 
# you will need to add commas in between the different values
best_beta = ...

In [ ]:
# run this cell to check your answers for common errors
check("tests/best-model.ok")

<div class="alert alert-warning">
<b>QUESTION:</b> What was your approach to finding a better model? Explain which variables you tried and why, as well as what metrics showed it was the "better". Reference the scatter plots, fit lines, RMSE, etc, and record the explanatory and response variables for your best model.
</div>

**ANSWER**: 

Finally, run the next cell to try your best model on the test data.

In [ ]:
# run this cell to try your best model on the test data.
# if you get an error, make sure you ran the cell where you saved
# the parameters of your best model
X_test = make_X(bike_test, best_expl_vars)
y_test = bike_test[best_response_var]

if not best_intercept:
    X_test.drop("intercept", axis=1, inplace=True)

# make predictions
pred_test = X_test @ best_beta

# generate plots 
f, [ax1, ax2] = plt.subplots(2, figsize=(12, 12))

sns.regplot(x=y_test, y=pred_test, ax=ax1, color="#ED4E33") 
ax1.set_xlabel(best_response_var)
ax1.set_ylabel("predicted {}".format(best_response_var))
ax1.set_title("Predicted vs. Actual Values (Test Data)")

sns.scatterplot(x=y_test, y=y_test - pred_test, ax=ax2, color="#ED4E33") 
ax2.set_xlabel(best_response_var)
ax2.set_ylabel("error ({} - predicted {})".format(best_response_var, best_response_var))
ax2.set_title("Error (Test Data)")
ax2.hlines(y=0, xmin=min(y_test), xmax=max(y_test))


# calculate rmse
print("Test data RMSE = {}".format(np.sqrt(mse(pred_test, y_test))))


<div class="alert alert-warning">
<b>QUESTION:</b> How did your model perform on the test data compared to how it performed on the training and validation data? Given what you know about how models "learn" from data, are the results you saw in line with your expectations?
</div>

**ANSWER:**

#### References
- Bike-Sharing data set from University of California Irvine's Machine Learning Repository https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset
- Portions of text and code adapted from Professor Jonathan Marshall's Legal Studies 190 (Data, Prediction, and Law) course materials: [lab 2-22-18, Linear Regression](https://github.com/ds-modules/LEGALST-190/tree/master/labs/2-22) (Author Keeley Takimoto)  and [lab 3-22-18, Exploratory Data Analysis](https://github.com/ds-modules/LEGALST-190/tree/masterlabs/3-22) (Author Keeley Takimoto)
- "Capital Bikeshare, Washington, DC" header image by [Leeann Caferatta](https://www.flickr.com/photos/leeanncafferata/34309356871) licensed under [CC BY-ND 2.0](https://creativecommons.org/licenses/by-nd/2.0/)